In [ ]:
import sys
sys.path.append('..')
import numpy as np
import math
import matplotlib.pylab as plt
import rti_msr_utils as rti
import particle_filter_class as pfc
import heapq


In [ ]:
num_CIR_samples = 50  # number of logged CIR samples
CIR_samp_period = 1/(2*(4*124.8e6))  # (s) sampling period for CIR measurement
anchor_UWB_Ids = [0, 1, 2, 3, 4]  # device Ids
SpeedofLight = 299792458  # (m/s) speed of light

In [ ]:
anchor_pair_list = []
for i in range(0,5):
    for j in range(i+1,5):
        anchor_pair_list.append((i,j))

![image](../pic/room1.png)

In [ ]:
# room 1
pos = [(2.28,0.9),(0,1.55),(5.09,6.52),(5.09,2.2),(0.2,5.69)]
room_length = 5.09
room_width = 10.02
rel_x = 1.25
rel_y = 1.98
# x_monitored = 2.45
# y_monitored = 4.40
x_monitored = room_length
y_monitored = room_width

In [ ]:
file_path = "../../data/uwb2/uwb2_exp002.csv"
uwb2_exp001, cir, tx_id_uwb, rx_id_uwb, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y = rti.load_csv_data(file_path, num_CIR_samples, tag_name_left="tag4422_", tag_name_right="tag89b3_")
cir_up = rti.upsample_and_align_cir(cir, fp, freq_s_ratio=64)
grouped_data = rti.group_data_by_tx_rx(tx_id_uwb, rx_id_uwb, cir, cir_up, rx_pream_count, fp, rx_lvl, time_stamp, ground_x, ground_y)

In [ ]:
# Constants
freq_s_ratio = 64
ALPHA_H = 0.05
BETA_B = 0.001
BETA_C = 0.1
MAX_COUNT = 10000
AMPLIFY = 1.3
WINDOW_SIZE = 2*freq_s_ratio
SEGMENT_THRESHOLD = 5

In [ ]:
min_heap = []
for anchor_pair, data in grouped_data.items():
    data['h'] = None
    data['var_b'] = None
    data['var_c'] = None
    for i, time_stamp in enumerate(data['time_stamp']):
        heapq.heappush(min_heap, (time_stamp, anchor_pair, i))


In [ ]:
import matplotlib.pyplot as plt

def plot_particles(particles, room_x_len, room_y_len, true_pos_x=None, true_pos_y=None):
    """
    Plot particles on a 2D grid along with the room dimensions.
    
    Parameters:
    particles (list): List of Particle objects.
    room_x_len (float): Length of the room along the x-axis.
    room_y_len (float): Length of the room along the y-axis.
    true_pos_x (float, optional): True x position to be plotted.
    true_pos_y (float, optional): True y position to be plotted.
    """
    # Extract x and y coordinates from particles
    x_pos = [particle.cx for particle in particles]
    y_pos = [particle.cy for particle in particles]
    
    # Initialize plot
    plt.figure(figsize=(10, 10))
    
    # Plot particles
    plt.scatter(x_pos, y_pos, c='blue', marker='o', label='Particles')
    
    # Plot true position if provided
    if true_pos_x is not None and true_pos_y is not None:
        plt.scatter(true_pos_x, true_pos_y, c='red', marker='x', label='True Position')
    
    # Set axis limits to match room dimensions
    plt.xlim(0, room_x_len)
    plt.ylim(0, room_y_len)
    
    # Add labels and title
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title('Particle Distribution in Room')
    
    # Add grid lines
    plt.grid(True)
    
    # Show legend
    plt.legend()
    
    # Show plot
    plt.show()

# Example usage:
# plot_particles(particles, 5.09, 10.02, true_pos_x=2.5, true_pos_y=5)


In [ ]:
# Define the step size for generating particles
# (You can adjust these based on how densely you want to populate the particles)
step_length = room_length / 50  # Step size along the length
step_width = room_width / 50  # Step size along the width

# Generate arrays for x and y positions
x_pos_array = np.arange(0, room_length, step_length)
y_pos_array = np.arange(0, room_width, step_width)

# Generate particles
sigmas = [10, 10]    # standard deviation of each attr of a particle
particles = [pfc.Particle(x, y, sigmas) for x in x_pos_array for y in y_pos_array]

# Function to plot particles (assuming you have one)
plot_particles(particles, 5.09, 10.02)

In [ ]:

# Process the heap
count = 0
last_time = None
while min_heap:
    # Exit if reached MAX_COUNT
    if count % 100 == 90:
        true_pos_x = grouped_data[anchor_pair]["ground_x"][index]
        true_pos_y = grouped_data[anchor_pair]["ground_y"][index]
        plot_particles(particles, 5.09, 10.02, true_pos_x, true_pos_y)
        print(f"true position: {true_pos_x}, {true_pos_y}")
    if count == MAX_COUNT:
        break

    # Pop the smallest time_stamp item
    smallest_time, anchor_pair, index = heapq.heappop(min_heap)

    # Print current state (for debugging)
    # print(f"Processing anchor pair {anchor_pair} with time stamp {smallest_time} at index {index}")
    # print(f"Item data: {grouped_data[anchor_pair]}")

    # Update count
    count += 1

    # Extract and update variables
    z = grouped_data[anchor_pair]['cir_up'][index]
    if grouped_data[anchor_pair]['h'] is None:
        grouped_data[anchor_pair]['h'] = z
        continue

    h = grouped_data[anchor_pair]['h'] = grouped_data[anchor_pair]['h'] * (1 - ALPHA_H) + z * ALPHA_H
    if grouped_data[anchor_pair]['var_b'] is None:
        delta = np.abs(z - h)
        grouped_data[anchor_pair]['var_b'] = delta
        grouped_data[anchor_pair]['var_c'] = delta
    else:
        grouped_data[anchor_pair]['var_b'] = grouped_data[anchor_pair]['var_b'] * (1 - BETA_B) + np.abs(z - h) * BETA_B
        grouped_data[anchor_pair]['var_c'] = grouped_data[anchor_pair]['var_c'] * (1 - BETA_C) + np.abs(z - h) * BETA_C    

    # Extract and preprocess data
    variance = grouped_data[anchor_pair]['var_c'] - AMPLIFY * grouped_data[anchor_pair]['var_b']
    data_length = len(variance)

    # Initialize position marker
    position_marker = -1  # Position where the condition is met

    # Define loop limit to avoid redundant calculations
    loop_limit = data_length - WINDOW_SIZE + 1

    # Search for the position where the condition is met
    for j in range(data_length):
        # Skip negative variance values
        if variance[j] < 0:
            continue

        # Avoid going out of bounds
        if j >= loop_limit:
            break

        # Calculate sum of the window and check against threshold
        window_sum = sum(variance[j:j + WINDOW_SIZE])
        if window_sum >= SEGMENT_THRESHOLD:
            position_marker = j
            break
    
    if position_marker == -1:
        continue #invalid position, skip weighting
    ################################
    #particle filtering steps
    ################################
    #3,1 Transition particles by gaussian distribution
    
    if last_time is None:
        last_time = smallest_time
    particles = pfc.transition_step(particles,sigmas,last_time - smallest_time)
    last_time = smallest_time
    #3.2 compute each particle's weight by it's distance to the predicted position
    distance = (position_marker - freq_s_ratio * 4) * 0.3 / freq_s_ratio
    weights = pfc.weighting_step(particles,distance,anchor_pair,pos)
    #3.4 resample the particles by their weight,remove particles whose weight are too low
    particles = pfc.resample_step(particles,weights,rsp_sigmas = [2,2])
    # Position marker now holds the first position where the condition is met or -1 if it's not met
print("Done!")